# Recurrent neural networks

In the previous module, we have been using rich **_semantic representations of text_**, and a simple linear classifier on top of the embeddings. What this architecture does is to capture aggregated meaning of words in a sentence, but it does not take into account the **order** of words, because aggregation operation on top of embeddings removed this information from the original text. Because these models are unable to model word ordering, they cannot solve more complex or ambiguous tasks such as text generation or question answering.

To capture the meaning of text sequence, we need to use another neural network architecture, which is called a **recurrent neural network**, or RNN. In RNN, we pass our sentence through the network one word vector from a news article sequence at a time, and the network produces some **state**, which we then pass to the network again with the next one word vector from the sequence.  RNN storing a "memory" of the previous in the state, helps the network understand the **_context of the sentence_** to be able to predict the network word in the sequence.

<img alt="Image showing an example recurrent neural network generation." src="images/5-recurrent-networks-1.png" align="middle" />

- Given the input sequence of word vectors $X_0,\dots,X_n$, RNN creates a sequence of neural network blocks, and trains this sequence end-to-end using back propagation. 
- Each network block takes a pair $(X_i,h_i)$ as an input, and produces $h_{i+1}$ as a result. 
- Final state $h_n$ or output $y$ goes into a linear classifier to produce the result. 
- All network blocks share the same weights, and are trained end-to-end using one backpropagation pass.

The hidden cell containing the current and prior state is calculated with the following formula:

- $h(t) = {tanh(W_{h}h_{t-1} + W_{x}x_{t} + B_{h}) }$ 
- $y(t) = {  W_{y}h_{t} + B_{y} }$ 
- Formula for calculating ${tanh}$ -  $f(x) = {\frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}} $

At each network block, weights $W_{x}$ are applied to the numeric word vector input value; applying the previous hidden state $W_{h}$; and the  final state $W_{y}$. The ${tanh}$ activation function is applied to the hidden layer to produce values between $[-1,1]$.

Because state vectors $h_0,\dots,h_n$ are passed through the network, it is able to learn the sequential dependencies between words. For example, when the word **_not_** appears somewhere in the sequence, it can learn to negate certain elements within the state vector, resulting in negation.  

Let's see how recurrent neural networks can help us classify our news dataset.

In [1]:
import torch
import torchtext
from torchinfo import summary
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Simple RNN classifier

In the case of simple RNN, each recurrent unit is a simple linear network, which takes concatenated input vector and state vector, and produce a new state vector. PyTorch represents this unit with `RNNCell` class, and a networks of each cells - as `RNN` layer.

To define an RNN classifier, we will first apply an embedding layer to lower the dimensionality of input vocabulary, and then have a RNN layer on top of it: 

In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** We use untrained embedding layer here for simplicity, but for even better results we can use pre-trained embedding layer with Word2Vec or GloVe embeddings, as described in the previous unit. For better understanding, you might want to adapt this code to work with pre-trained embeddings.

In our case, we will use padded data loader, so each batch will have a number of padded sequences of the same length. RNN layer will take the sequence of embedding tensors, and produce two outputs:

* The `input` to the embedding layer is the word sequence or news article
* The `embedding layer` output contains the vector index value in vocab for each word in the sequence
* $x$ is a sequence of RNN cell outputs at each step.  
* $h$ is a final `hidden state` for the last element of the sequence.  Each RNN hidden layer stores the prior word in the sequence and the current as each word in the sequence is passed through the layers.

We then apply a fully-connected linear classifier to get the probability for number of classes.

> **Note:** RNNs are quite difficult to train, because once the RNN cells are unrolled along the sequence length, the resulting number of layers involved in back propagation is quite large. Thus we need to select small learning rate, and train the network on larger dataset to produce good results. It can take quite a long time, so using GPU is preferred.

In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3340625
6400: acc=0.4190625
9600: acc=0.48
12800: acc=0.52171875
16000: acc=0.5511875
19200: acc=0.5776041666666667
22400: acc=0.5994196428571429
25600: acc=0.6203125
28800: acc=0.6385416666666667
32000: acc=0.65396875
35200: acc=0.6679829545454545
38400: acc=0.6797395833333333
41600: acc=0.6912740384615385
44800: acc=0.7015848214285715
48000: acc=0.7107291666666666
51200: acc=0.71904296875
54400: acc=0.72625
57600: acc=0.7325173611111111
60800: acc=0.7388322368421053
64000: acc=0.74484375
67200: acc=0.7508630952380952
70400: acc=0.7560795454545455
73600: acc=0.7609510869565217
76800: acc=0.7654036458333333
80000: acc=0.7696625
83200: acc=0.7738341346153846
86400: acc=0.7775231481481482
89600: acc=0.780625
92800: acc=0.784084051724138
96000: acc=0.7874895833333333
99200: acc=0.790241935483871
102400: acc=0.793046875
105600: acc=0.7957102272727272
108800: acc=0.7985477941176471
112000: acc=0.8011696428571429
115200: acc=0.8034722222222223
118400: acc=0.8055827702702703


(0.03306732177734375, 0.806625)

Now, let's load the test dataset to evaluate the trained RNN model.  We'll be using the 4 different classes of the news categories to map the predicted output with the targeted label.

In [4]:
print(f'class map: {classes}')

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, collate_fn=padify, shuffle=True)

class map: ['World', 'Sports', 'Business', 'Sci/Tech']


Before we evaluate the model, we'll extract the padded vector dataset from the dataloader.  We will use the vocab.itos function to convert the numeric index to the word it matches in the vocabulary.  When conversion from numeric to string happens for padded vectors, the '0' values are set to a special character `<unk>` as an unknown identifier. So, the character needs to be removed, depending on the unknown values from the padded zeros.

Finally, we’ll run the model with our test dataset to verify if the expected output matched the predicted.

In [5]:
net.eval()

with torch.no_grad():
    for batch_idx, (target, data) in enumerate(test_loader):
        
        word_lookup = [vocab.itos[w] for w in data[batch_idx]]
        unknow_vals = {'<unk>'}
        word_lookup = [ele for ele in word_lookup if ele not in unknow_vals]
        print('Input text:\n {}\n'.format(word_lookup))
        
        data, target = data.to(device), target.to(device)
        pred = net(data)
        print(torch.argmax(pred[batch_idx]))
        print("Actual:\nvalue={}, class_name= {}\n".format(target[batch_idx], classes[target[batch_idx]]))
        print("Predicted:\nvalue={}, class_name= {}\n".format(pred[0].argmax(0),classes[pred[0].argmax(0)]))
        break

Input text:
 ['business', 'bush', 'administration', 'washington', 'post', 'business', 'columnist', 'steven', 'pearlstein', 'will', 'be', 'online', 'to', 'discuss', 'his', 'latest', 'column', ',', 'which', 'looks', 'at', 'the', 'bush', 'administration', "'", 's', 'plans', 'for', 'social', 'security', 'and', 'health', 'care', '.']

tensor(2)
Actual:
value=2, class_name= Business

Predicted:
value=2, class_name= Business



## Long Short Term Memory (LSTM)

One of the main problems of classical RNNs is the so-called **vanishing gradients** problem. Because RNNs are trained end-to-end in one back-propagation pass, it is having hard times propagating error to the first layers of the network, and thus the network cannot learn relationships between distant tokens. The gradient helps in adjusting the weights during back-progagation to achieve better accuracy and reduce the error margin.  If the weights are too small the network does not learn.  Since the gradient decreases during back-propagation in RNNs, the network does not learn the initial inputs in the network.  In other ways, the network "forgets" the earlier word inputs.

One of the ways to avoid this problem is to introduce **explicit state management** by using so called **gates**. There are two most known architectures of this kind: **Long Short Term Memory** (LSTM) and **Gated Relay Unit** (GRU).

<img alt="Image showing an example long short term memory cell" src="images/5-recurrent-networks-2.png" align="middle" />


LSTM Network is organized in a manner similar to RNN, but there are two states that are being passed from layer to layer: `actual state` $c$, and `hidden vector` $h$. 
- At each unit, hidden vector $h_i$ is concatenated with input $x_i$, and they `control what happens to the state $c$ via **gates**. 
- Each gate is a neural network with `sigmoid` (${\sigma}$) activation (output in the range $[0,1]$), which can be thought of as bitwise mask when multiplied by the state vector. 

There are the following gates (from left to right on the picture above):
* **forget gate** takes hidden vector and determines, which components of the vector $c$ we need to forget, and which to pass through. The gate determines which words are not important and sigmod values close to zero need to be thrown out.  The formula is $f_t = \sigma(W_f  * [x_t + h_{t-1}] + b_f)$.
* **input gate** takes some information from the input and hidden vector, and inserts it into state. The formula for the input gate is a product of the new information $i_t = \sigma(W_i  * [x_t + h_{t-1}] + b_f)$ and the hidden  $\tilde{C_t} = \tanh(W_f  * [x_t + h_{t-1}] + b_f)$
* **output gate** transforms state via some linear layer with `tanh` activation, then selects some of its components using hidden vector $h_i$ to produce new state $c_{i+1}$.  The formula for the input gate is $o_t = \sigma(W_o  * [x_t + h_{t-1}] + b_f)$ and the hidden is ${h_t} = {o_t} * \tanh(C_t) $
* **cell state**  takes a product of the hidden state and the forget gate.  Then sums value with the product of the input gate and output gate. $C_t = f_t \circ C_{t-1} + i_t \circ \tilde{C_t}$ 

Components of the state $c$ can be thought of as some flags that can be switched on and off. For example, when we encounter a name *'Alice'* in the sequence, we may want to assume that it refers to female character, and raise the flag in the state that we have female noun in the sentence. When we further encounter phrases *and Tom*, we will raise the flag that we have plural noun. Thus by manipulating state we can supposedly keep track of grammatical properties of sentence parts.

> **Note**: A great resource for understanding internals of LSTM is this great article "Understanding LSTM Networks" by Christopher Olah.

While internal structure of LSTM cell may look complex, PyTorch hides this implementation inside the `LSTMCell` class, and provides a `LSTM` object to represent the whole LSTM layer. Thus, implementation of LSTM classifier will be pretty similar to the simple RNN which we have seen above:

In [6]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

Now let's train our network. Note that training LSTM is also quite slow, and you may not seem much raise in accuracy in the beginning of training. Also, you may need to play with `lr` learning rate parameter to find the learning rate that results in reasonable training speed.

In [7]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.2509375
6400: acc=0.264375
9600: acc=0.27979166666666666
12800: acc=0.3003125
16000: acc=0.32375
19200: acc=0.3491666666666667
22400: acc=0.3639732142857143
25600: acc=0.38234375
28800: acc=0.39788194444444447
32000: acc=0.41425
35200: acc=0.42798295454545454
38400: acc=0.44244791666666666
41600: acc=0.4593269230769231
44800: acc=0.47261160714285716
48000: acc=0.4881875
51200: acc=0.50271484375
54400: acc=0.5168933823529411
57600: acc=0.5295486111111111
60800: acc=0.5411019736842105
64000: acc=0.552671875
67200: acc=0.5638244047619048
70400: acc=0.5748153409090909
73600: acc=0.585
76800: acc=0.59453125
80000: acc=0.60405
83200: acc=0.6122956730769231
86400: acc=0.6208564814814815
89600: acc=0.6287053571428571
92800: acc=0.6360991379310345
96000: acc=0.6432916666666667
99200: acc=0.6497983870967742
102400: acc=0.65572265625
105600: acc=0.6617708333333333
108800: acc=0.6674264705882353
112000: acc=0.6726785714285715
115200: acc=0.6782986111111111
118400: acc=0.6834375


(0.04554988606770834, 0.686025)

## Packed sequences

In our example, we had to pad all sequences in the minibatch with zero vectors. While it results in some memory waste, with RNNs it is more critical that additional RNN cells are created for the padded input items, which take part in training, yet do not carry any important input information. It would be much better to train RNN only to the actual sequence size.

To do that, a special format of padded sequence storage is introduced in PyTorch. Suppose we have input padded minibatch which looks like this:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Here 0 represents padded values, and the actual length vector of input sequences is `[5,3,1]`.

In order to effectively train RNN with padded sequence, we want to begin training first group of RNN cells with large minibatch (`[1,6,9]`), but then end processing of third sequence, and continue training with shorted minibatches (`[2,7]`, `[3,8]`), and so on. Thus, packed sequence is represented as one vector - in our case `[1,6,9,2,7,3,8,4,5]`, and length vector (`[5,3,1]`), from which we can easily reconstruct the original padded minibatch.

To produce packed sequence, we can use `torch.nn.utils.rnn.pack_padded_sequence` function. All recurrent layers, including RNN, LSTM and GRU, support packed sequences as input, and produce packed output, which can be decoded using `torch.nn.utils.rnn.pad_packed_sequence`.

To be able to produce packed sequence, we need to pass length vector to the network, and thus we need a different function to prepare minibatches:

In [8]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)
test_loader_len = torch.utils.data.DataLoader(test_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

The actual network would be very similar to `LSTMClassifier` above, but `forward` pass will receive both padded minibatch and the vector of sequence lengths. After computing the embedding, we compute packed sequence, pass it to LSTM layer, and then unpack the result back.

> **Note**: We actually do not use unpacked result `x`, because we use output from the hidden layers in the following computations. Thus, we can remove the unpacking altogether from this code. The reason we place it here is for you to be able to modify this code easily, in case you should need to use network output in further computations.

In [9]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        _,(h,c) = self.rnn(pad_x)
        return self.fc(h[-1])

Now let's train our network with the padded sequence:

In [10]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.29875
6400: acc=0.359375
9600: acc=0.41458333333333336
12800: acc=0.462890625
16000: acc=0.50425
19200: acc=0.5391666666666667
22400: acc=0.5695089285714285
25600: acc=0.5971875
28800: acc=0.6198611111111111
32000: acc=0.6396875
35200: acc=0.6576988636363637
38400: acc=0.6736197916666666
41600: acc=0.6871394230769231
44800: acc=0.6997098214285714
48000: acc=0.70975
51200: acc=0.71970703125
54400: acc=0.727922794117647
57600: acc=0.7359375
60800: acc=0.7429934210526316
64000: acc=0.74978125
67200: acc=0.7558928571428571
70400: acc=0.7613920454545454
73600: acc=0.7664402173913043
76800: acc=0.7716536458333333
80000: acc=0.7760125
83200: acc=0.7801802884615384
86400: acc=0.7843981481481481
89600: acc=0.788125
92800: acc=0.7916810344827586
96000: acc=0.79515625
99200: acc=0.7982762096774193
102400: acc=0.801162109375
105600: acc=0.8040151515151515
108800: acc=0.8068198529411764
112000: acc=0.8093125
115200: acc=0.8118055555555556
118400: acc=0.8141807432432432


(0.029937251790364584, 0.8152833333333334)

> **Note:** You may have noticed the parameter `use_pack_sequence` that we pass to the training function. Currently, `pack_padded_sequence` function requires length sequence tensor to be on CPU device, and thus training function needs to avoid moving the length sequence data to GPU when training. You can look into implementation of `train_epoch_emb` helper function in the `torchnlp.py` file located in the local directory.

In [11]:
net.eval()

with torch.no_grad():
    for label,text,off in test_loader_len:
        
        text, label = text.to(device), label.to(device)
        off = off.to('cpu')
        print(f'off value: {off}')
        pred = net(text, off )
        print(f'target {label}')
        y=torch.argmax(pred, dim=1)
        print(f'pred: {y}')
        print("Predicted:\nvalue={}, class_name= {}\n".format(y[0],classes[y[0]]))
        print("Target:\nvalue={}, class_name= {}\n".format(label[0],classes[label[0]]))
        break
     

off value: tensor([39, 54, 44, 49, 31, 34, 45, 29, 47, 48, 37, 39, 27, 37, 35, 47])
label: tensor([2, 2, 2, 2, 0, 3, 1, 1, 2, 3, 3, 1, 3, 1, 2, 1])
pred: tensor([2, 2, 2, 2, 0, 3, 1, 1, 2, 3, 3, 1, 0, 1, 2, 1])
Predicted:
value=2, class_name= Business

Target:
value=2, class_name= Business



## Bidirectional and multilayer RNNs

In our examples, all recurrent networks operated in one direction, from beginning of a sequence to the end. It looks natural, because it resembles the way we read and listen to speech. However, since in many practical cases we have random access to the input sequence, it might make sense to run recurrent computation in both directions. Such networks are call **bidirectional** RNNs, and they can be created by passing `bidirectional=True` parameter to RNN/LSTM/GRU constructor.

> **Example**:   _self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True, bidrectional=True)_

When dealing with bidirectional network, we would need two hidden state vectors, one for each direction. PyTorch encodes those vectors as one vector of twice larger size, which is quite convenient, because you would normally pass the resulting hidden state to fully-connected linear layer, and you would just need to take this increase in size into account when creating the layer.

Recurrent network, one-directional or bidirectional, captures certain patterns within a sequence, and can store them into state vector or pass into output. As with convolutional networks, we can build another recurrent layer on top of the first one to capture higher level patterns, build from low-level patterns extracted by the first layer. This leads us to the notion of **multi-layer RNN**, which consists of two or more recurrent networks, where output of the previous layer is passed to the next layer as input.

<img alt="Image showing a Multilayer long-short-term-memory- RNN" src="images/5-recurrent-networks-3.jpg" align="middle" />

*Picture from "From a LSTM cell to a Multilayer LSTM Network with PyTorch" article by Fernando López*

PyTorch makes constructing such networks an easy task, because you just need to pass `num_layers` parameter to RNN/LSTM/GRU constructor to build several layers of recurrence automatically. This would also mean that the size of hidden/state vector would increase proportionally, and you would need to take this into account when handling the output of recurrent layers.

## RNNs for other tasks

In this unit, we have seen that RNNs can be used for sequence classification, but in fact, they can handle many more tasks, such as text generation, machine translation, and more. We will consider those tasks in the next unit.